# 宏观因子资产配置研究

更新日志
- 2022.10.22 
    - 梳理工作内容并搭建研究框架

In [1]:
import os, sys, argparse, logging

# 设置logging格式和级别
logging.basicConfig(
    level=logging.DEBUG,
    # level=logging.INFO, 
    format='%(filename)s-line%(lineno)d %(levelname)s: %(message)s'
    )

logging.disable(logging.DEBUG)
logging.disable(logging.INFO)

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from datetime import datetime
from copy import deepcopy

logging.disable(logging.NOTSET)

# 首先将framework的目录加进sys.path中，方便使用相对路径导入
framework_path = os.path.join(os.path.abspath(''), '../../../../国君研究所/工作/FOF_portfolio_toolbox/framework')
if framework_path not in sys.path:
    sys.path.append(framework_path)

this_path = os.path.abspath('.')
if this_path not in sys.path:
    sys.path.append(this_path)

from factor_portfolio import FactorExposureAlg, MvoAlg


In [2]:
from component.asset.asset import Asset

from import_func import getSvc
date_svc = getSvc('DateSvc')
raw_data_svc = getSvc('LxwWinddbRawDataSvc')
constant_svc = getSvc('ConstantSvc')

date_svc.setTradeDays(raw_data_svc.getTradeDays())

init YamlSvc
init DateSvc


lxw_winddb_sql_raw_data_svc.py-line69 DEBUG: raw_data_svc.sqlQuery: SELECT TRADE_DAYS FROM asharecalendar WHERE S_INFO_EXCHMARKET='SSE' ORDER BY TRADE_DAYS


init YamlSvc
init LxwWinddbRawDataSvc
init MysqlDbConnectorSvc
init ConstantSvc


date_svc.py-line31 INFO: DateSvc: set trade days


## 1. 选择待研究的大类资产(国内大类资产)

|No.    |资产类别       |资产标的       |权重约束   |
|--     |--             |--            |--         |
|1      |大市值股票     |沪深300指数     ||
|2      |中小市值股票   |中证500指数	   |
|3      |利率债         |中债国债总指数  ||
|4      |信用债         |中债信用债总指数|	|
|5      |商品	        |南华综合指数	||
|6      |黄金	        |Au9999	||

### 1.1 读入资产原始数据
所有数据存放在../data/assets中

In [3]:
# 读入资产原始数据
asset_raw_data_path = os.path.join(this_path, '../data/assets')
asset_dict = {}
for file_name in os.listdir(asset_raw_data_path):
    # 读入原始数据
    asset_name = file_name.split('.')[0]
    raw_data = pd.read_excel(os.path.join(asset_raw_data_path,file_name), index_col=0)
    raw_nav_data = raw_data['收盘价']
    raw_nav_data.index.name = 'date'
    
    # 创建资产实体并加入dict
    asset_obj = Asset(asset_name)
    asset_obj.setRawNavData(raw_nav_data)
    
    asset_dict[asset_name] = asset_obj

logging.debug(asset_dict)

1532541054.py-line17 DEBUG: {'Au9999': <component.asset.asset.Asset object at 0x0000020BB5688DC0>, '中债信用债总指数': <component.asset.asset.Asset object at 0x0000020BFFF9AFD0>, '中债国债总指数': <component.asset.asset.Asset object at 0x0000020B9010AC10>, '中证500指数': <component.asset.asset.Asset object at 0x0000020B9035B640>, '南华综合指数': <component.asset.asset.Asset object at 0x0000020B9024E460>, '沪深300指数': <component.asset.asset.Asset object at 0x0000020B900D3C40>}


## 2. 构造宏观因子组合

### 2.1 读入因子原始数据

In [4]:
# 载入因子
factor_raw_data_path = os.path.join(this_path, '../data/factors')
factor_dict = {}
for file_name in os.listdir(factor_raw_data_path):
    # 读入原始数据
    factor_name = file_name.split('.')[0]
    raw_data = pd.read_excel(os.path.join(factor_raw_data_path,file_name), index_col=0)
    raw_data.index.name = 'date'
    raw_data = raw_data.iloc[:, 0]

    # 去除第一行空值, 去除0值防止除0错误
    raw_data = raw_data.iloc[1:]
    raw_data.loc[raw_data==0] = 1e-6
    
    # 创建因子实体并加入dict
    factor_obj = Asset(factor_name)
    factor_obj.setRawNavData(raw_data)
    
    factor_dict[factor_name] = factor_obj

logging.debug(factor_obj)

3414513511.py-line21 DEBUG: <component.asset.asset.Asset object at 0x0000020B905782E0>


### 2.2 宏观因子组合

In [5]:
# 最大化sharp比率
mvo_alg = MvoAlg('mvo')
id_date = datetime(2020, 1, 5)

# 向前取多少数据用于计算
look_back = constant_svc.DAY_OF_YEAR

# 为因子设置日期
for factor_obj in factor_dict.values():
    factor_obj.setIdDate(id_date, look_back)

factor_data = {factor: factor_obj.getUsableNavData() for factor, factor_obj in factor_dict.items()}
# logging.debug(factor_data)

factor_portfolio = mvo_alg.run(factor_data, returns_data=True, max_sharp=True, verbose=True)
logging.debug(factor_portfolio)


alg_base.py-line19 INFO: alg mvo: init args
factor_portfolio.py-line48 DEBUG: mu = growth    -1.000000e+00
real      1.135780e+114
dtype: float64
factor_portfolio.py-line49 DEBUG: s =             growth       real
growth  418.579698   1.799552
real      1.799552  28.180601


                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Oct 23 12:00:01 PM: Your problem has 3 variables, 5 constraints, and 0 parameters.
(CVXPY) Oct 23 12:00:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 23 12:00:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 23 12:00:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 23 12:00:01 PM: Compiling problem (target solver=OSQP).
(CVXPY) Oct 23 12:00:01 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing 

SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

## 3. 配置因子VS配置资产（模型性能比较）